In [1]:
import pandas as pd
import numpy as np


from openprompt.data_utils.utils import InputExample
from openprompt.data_utils.data_processor import DataProcessor
from sklearn.model_selection import train_test_split
## self-modified
from pipeline_base_modified import PromptDataLoader, PromptForClassification
from template_generation_wzy import LMBFFTemplateGenerationTemplate, T5TemplateGenerator
from openprompt.plms import load_plm
from tqdm import tqdm
import torch.nn as nn
import os
from openprompt.prompts import ManualVerbalizer
from openprompt.prompts import ManualTemplate
from openprompt.trainer import ClassificationRunner
import copy
import torch
from transformers import  AdamW, get_linear_schedule_with_warmup


from openprompt import PromptForClassification
from openprompt.plms import load_plm
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, classification_report
from openprompt.prompts import ManualTemplate



/home/zhiyuan/ENTER/lib/python3.9/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [2]:
def sum_list(a,b):
    res = []
    if len(a) >= len(b):
        for i in range(len(b)):
            res.append(a[i]+b[i])# 
        for i in range(len(a) - len(b)):
            res.append(a[len(b)+i])
        return res
    elif len(a) < len(b):
        return sum_list(b, a)


os.environ['CUDA_VISIBLE_DEVICES'] = '1'
use_cuda = True
plm, tokenizer, model_config, WrapperClass = load_plm("roberta", "roberta-large")


In [3]:
Personalities = ['A','C','E','O','N']

for personality in Personalities:
    with open('label_words/' + personality + '_words.txt', 'r') as f:
        pos = f.readline().split(',')
        neg = f.readline().split(',')

    with open('label_words/' + personality + '_weights.txt', 'r') as f:
        pos_weights = eval(f.readline())
        neg_weights = eval(f.readline())
    
    
    candidate_templates = []
    datasets = ['Friends', 'MyPersonality', 'Essay', 'Pan']
    for dataset in datasets:
        with open('templates/Adapted_t5_large_'+dataset+'_'+personality+'_templates_top_10.txt', 'r') as f:
            candidate_templates += [i.strip() for i in f.readlines()]
    
    classes = [0,1]        
    myverbalizer = ManualVerbalizer(
        classes = classes,
        label_words = {
            0 : neg, 
            1 : pos
        },
        tokenizer=tokenizer)

    from openprompt import PromptDataLoader
    batch_size = 1

    overall_logits = []

    for i in range(len(candidate_templates)):
        sample = InputExample(guid=i, text_a="", label=0)
        mytemplate = ManualTemplate(
            text = candidate_templates[i],
            tokenizer = tokenizer,
        )
        prompt_model = PromptForClassification(plm=plm, template=mytemplate, verbalizer=myverbalizer, freeze_plm=True)
        if use_cuda:
            prompt_model = prompt_model.cuda()
        prompt_model.eval()
        validation_dataloader = PromptDataLoader(dataset=[sample], template=mytemplate, tokenizer=tokenizer,
            tokenizer_wrapper_class=WrapperClass, max_seq_length=128,
            batch_size=batch_size, shuffle=False, teacher_forcing=False, predict_eos_token=False,
            truncate_method="head")

        for step, inputs in enumerate(validation_dataloader):
            if use_cuda:
                inputs = inputs.cuda()
            logits = prompt_model.forward_without_verbalize(inputs)
            label_words_logits = prompt_model.verbalizer.project(logits)
            label_words_probs = prompt_model.verbalizer.normalize(label_words_logits)
            label_words_logits = label_words_probs # torch.log(label_words_probs+1e-15)
            overall_logits.append(label_words_logits.detach())

    overall_logits = torch.cat(overall_logits)
    overall_logits = torch.mean(overall_logits, 0)

    neg_logits = [0.1*float(i) for i in list(overall_logits[0]/overall_logits[0].sum().cpu())]
    pos_logits = [0.1*float(i) for i in list(overall_logits[1]/overall_logits[1].sum().cpu())]

    pos_weights_ = [i/sum(pos_weights) for i in pos_weights]
    neg_weights_ = [i/sum(neg_weights) for i in neg_weights]


    pos_logit_weight = sum_list(pos_weights_, pos_logits)
    neg_logit_weight = sum_list(neg_weights_, neg_logits)



    pos_dict = {}
    for word, weight in zip(pos, pos_logit_weight[:len(pos)]):
        pos_dict[word] = weight

    neg_dict = {}
    for word, weight in zip(neg, neg_logit_weight[:len(neg)]):
        neg_dict[word] = weight  

    n = 200
    pos_ = sorted(pos_dict.items(), key=lambda kv:(kv[1], kv[0]), reverse=True)[:n]
    neg_ = sorted(neg_dict.items(), key=lambda kv:(kv[1], kv[0]), reverse=True)[:n]
    
    print('Top', n, 'positive label words are:', [i[0] for i in pos_])

    print('Top', n, 'negative label words are:', [i[0] for i in neg_])

    with open('label_words/posterior_'+personality+'_words.txt', 'w') as f:
        f.write(','.join([i[0] for i in pos_]))
        f.write('\n')
        f.write(','.join([i[0] for i in neg_]))

    with open('label_words/posterior_'+personality+'_weights.txt', 'w') as f:
        f.write(str([10*i[1] for i in pos_]))
        f.write('\n')
        f.write(str([-10*i[1] for i in neg_]))





tokenizing: 1it [00:00, 2089.84it/s]
tokenizing: 1it [00:00, 2090.88it/s]
tokenizing: 1it [00:00, 2269.65it/s]
tokenizing: 1it [00:00, 2465.79it/s]
tokenizing: 1it [00:00, 2044.01it/s]
tokenizing: 1it [00:00, 2105.57it/s]
tokenizing: 1it [00:00, 2317.30it/s]
tokenizing: 1it [00:00, 2813.08it/s]
tokenizing: 1it [00:00, 2421.65it/s]
tokenizing: 1it [00:00, 1952.66it/s]
tokenizing: 1it [00:00, 1672.37it/s]
tokenizing: 1it [00:00, 2444.23it/s]
tokenizing: 1it [00:00, 2424.45it/s]
tokenizing: 1it [00:00, 2107.69it/s]
tokenizing: 1it [00:00, 1721.09it/s]
tokenizing: 1it [00:00, 1855.89it/s]
tokenizing: 1it [00:00, 1822.03it/s]
tokenizing: 1it [00:00, 2119.41it/s]
tokenizing: 1it [00:00, 2713.00it/s]
tokenizing: 1it [00:00, 2077.42it/s]
tokenizing: 1it [00:00, 2752.17it/s]
tokenizing: 1it [00:00, 2720.04it/s]
tokenizing: 1it [00:00, 2590.68it/s]
tokenizing: 1it [00:00, 2543.54it/s]
tokenizing: 1it [00:00, 1938.22it/s]
tokenizing: 1it [00:00, 2036.07it/s]
tokenizing: 1it [00:00, 2235.77it/s]
t

Top 200 positive label words are: ['good-humoured', 'onefold', 'likeable', 'nice', 'hot', 'sadistic', 'kind', 'freehearted', 'sympathetic', 'likable', 'congenial', 'easy', 'derivative', 'fine', 'warmly', 'warm', 'easy-going', 'simpleton', 'simple-minded', 'simple', 'lukewarm', 'great', 'polite', 'courteous', 'civil', 'gracious', 'polished', 'gallant', 'affable', 'helpful', 'prepared', 'heat', 'genteel', 'baking', 'warmth', 'aflutter', 'sincere', 'warm hearted', 'solemn', 'tisane', 'sentimental', 'liqueur', 'large-minded', 'heartfelt', 'elixir', 'cordial', 'cheer', 'invigorating', 'enjoyable', 'serious-minded', 'patient of', 'nesh', 'sensitive', 'medium', 'sensible', 'aware', 'tolerant', 'caring', 'beneficial', 'charitable', 'agreeable', 'resistant', 'welfare', 'humanitarian', 'eleemosynary', 'merciful', 'conformable', 'pleasant', 'happy', 'smooth', 'hard-nosed', 'pleasing', 'free-and-easy', 'patient', 'peaceful', 'patriotic', 'affected role', 'loyal', 'trustful', 'womanly', 'sweet', 'f

tokenizing: 1it [00:00, 2585.88it/s]
tokenizing: 1it [00:00, 2537.39it/s]
tokenizing: 1it [00:00, 3374.34it/s]
tokenizing: 1it [00:00, 2522.13it/s]
tokenizing: 1it [00:00, 2704.26it/s]
tokenizing: 1it [00:00, 2130.17it/s]
tokenizing: 1it [00:00, 2059.06it/s]
tokenizing: 1it [00:00, 2441.39it/s]
tokenizing: 1it [00:00, 1734.62it/s]
tokenizing: 1it [00:00, 2542.00it/s]
tokenizing: 1it [00:00, 2835.91it/s]
tokenizing: 1it [00:00, 2880.70it/s]
tokenizing: 1it [00:00, 2595.49it/s]
tokenizing: 1it [00:00, 2624.72it/s]
tokenizing: 1it [00:00, 2355.03it/s]
tokenizing: 1it [00:00, 2143.23it/s]
tokenizing: 1it [00:00, 1952.66it/s]
tokenizing: 1it [00:00, 1853.43it/s]
tokenizing: 1it [00:00, 2153.13it/s]
tokenizing: 1it [00:00, 2062.10it/s]
tokenizing: 1it [00:00, 2166.48it/s]
tokenizing: 1it [00:00, 2376.38it/s]
tokenizing: 1it [00:00, 2582.70it/s]
tokenizing: 1it [00:00, 2100.30it/s]
tokenizing: 1it [00:00, 1840.41it/s]
tokenizing: 1it [00:00, 1916.96it/s]
tokenizing: 1it [00:00, 1731.75it/s]
t

Top 200 positive label words are: ['truehearted', 'sadistic', 'organized', 'organised', 'accurate', 'on time', 'sad', 'precise', 'exact', 'sweetheart', 'hard work', 'hard', 'hard-and-fast', 'hardworking', 'hard working', 'right winger', 'fastidious', 'hard-nosed', 'authentic', 'secure', 'reliable', 'canny', 'exacting', 'logical', 'brief', 'straightaway', 'acquisitive', 'consistent', 'measured', 'concerned', 'legitimate', 'careful', 'succinct', 'concise', 'prompt', 'coherent', 'attentive', 'command prompt', 'anxious', 'lucid', 'mindful', 'terse', 'motivate', 'solicitous', 'work', 'strong', 'positive', 'regular', 'reproducible', 'steady', 'meticulous', 'painstaking', 'steadily', 'brace', 'punctilious', 'unfluctuating', 'unfaltering', 'decided', 'decisive', 'conclusive', 'reckful', 'simpleton', 'simple-minded', 'simple', 'difficult', 'tauten', 'free', 'timely', 'solid', 'productive', 'responsible for', 'responsible', 'company', 'firm', 'enterprise', 'punctual', 'corporation', 'unwaveringl

tokenizing: 1it [00:00, 2065.14it/s]
tokenizing: 1it [00:00, 2764.87it/s]
tokenizing: 1it [00:00, 2603.54it/s]
tokenizing: 1it [00:00, 2584.29it/s]
tokenizing: 1it [00:00, 2598.70it/s]
tokenizing: 1it [00:00, 2692.11it/s]
tokenizing: 1it [00:00, 1860.00it/s]
tokenizing: 1it [00:00, 2095.06it/s]
tokenizing: 1it [00:00, 2053.01it/s]
tokenizing: 1it [00:00, 1769.75it/s]
tokenizing: 1it [00:00, 2344.50it/s]
tokenizing: 1it [00:00, 1932.86it/s]
tokenizing: 1it [00:00, 2162.01it/s]
tokenizing: 1it [00:00, 2125.85it/s]
tokenizing: 1it [00:00, 2149.82it/s]
tokenizing: 1it [00:00, 2752.17it/s]
tokenizing: 1it [00:00, 2057.04it/s]
tokenizing: 1it [00:00, 2098.20it/s]
tokenizing: 1it [00:00, 1860.83it/s]
tokenizing: 1it [00:00, 1846.90it/s]
tokenizing: 1it [00:00, 2654.62it/s]
tokenizing: 1it [00:00, 2002.05it/s]
tokenizing: 1it [00:00, 2214.52it/s]
tokenizing: 1it [00:00, 2147.62it/s]
tokenizing: 1it [00:00, 2132.34it/s]
tokenizing: 1it [00:00, 2070.24it/s]
tokenizing: 1it [00:00, 2770.35it/s]
t

Top 200 positive label words are: ['happy', 'happy-go-lucky', 'good-humoured', 'truehearted', 'sadistic', 'long winded', 'bigmouthed', 'loud', 'brazen', 'extraverted', 'noisy', 'strong-growing', 'funny', 'noise', 'extravertive', 'vocal', 'sounded', 'shout', 'extroverted', 'extrospective', 'extrorse', 'talkative', 'drivellous', 'chatty', 'verbose', 'garrulous', 'prolix', 'loquacious', 'aggressive', 'dauntless', 'bellicose', 'daredevil', 'bad-mannered', 'sociable', 'exciting', 'social', 'going strong', 'up-and-coming', 'nice', 'likeable', 'boldface', 'bold', 'non finite verb', 'verbal', 'bluff', 'audacious', 'assertive', 'lectic', 'sicker', 'enthusiastic', 'self assured', 'confident', 'sure-footed', 'hot', 'attractive', 'mettlesome', 'enlivened', 'spirited', 'bouncing', 'communicative', 'hard work', 'hard', 'appealing', 'simple', 'energetic', 'charismatic', 'magnetic', 'magnetised', 'free', 'gay', 'cute', 'easy-going', 'shiny', 'boldness', 'rambunctious', 'daring', 'straight-from-the-sho

tokenizing: 1it [00:00, 2468.69it/s]
tokenizing: 1it [00:00, 1559.22it/s]
tokenizing: 1it [00:00, 2605.16it/s]
tokenizing: 1it [00:00, 2019.40it/s]
tokenizing: 1it [00:00, 1552.87it/s]
tokenizing: 1it [00:00, 1536.94it/s]
tokenizing: 1it [00:00, 1531.33it/s]
tokenizing: 1it [00:00, 2673.23it/s]
tokenizing: 1it [00:00, 2244.14it/s]
tokenizing: 1it [00:00, 1428.09it/s]
tokenizing: 1it [00:00, 2498.10it/s]
tokenizing: 1it [00:00, 2187.95it/s]
tokenizing: 1it [00:00, 2734.23it/s]
tokenizing: 1it [00:00, 2761.23it/s]
tokenizing: 1it [00:00, 2195.97it/s]
tokenizing: 1it [00:00, 2039.04it/s]
tokenizing: 1it [00:00, 2058.05it/s]
tokenizing: 1it [00:00, 2305.83it/s]
tokenizing: 1it [00:00, 1752.74it/s]
tokenizing: 1it [00:00, 2250.16it/s]
tokenizing: 1it [00:00, 2134.51it/s]
tokenizing: 1it [00:00, 2744.96it/s]
tokenizing: 1it [00:00, 2137.77it/s]
tokenizing: 1it [00:00, 1705.69it/s]
tokenizing: 1it [00:00, 2193.67it/s]
tokenizing: 1it [00:00, 2525.17it/s]
tokenizing: 1it [00:00, 2111.94it/s]
t

Top 200 positive label words are: ['good-humoured', 'complicated', 'great', 'sad', 'sadistic', 'truehearted', 'well-informed', 'free', 'intelligent', 'level-headed', 'silly', 'hardworking', 'hard working', 'smart', 'funny', 'atypical', 'complex', 'likeable', 'intellectual', 'cerebral', 'voguish', 'freehearted', 'ache', 'building complex', 'bad-mannered', 'philosophical', 'philosophic', 'brilliant', 'deep', 'subtle', 'selfish', 'bright', 'promising', 'low', 'heavy', 'innovative', 'forward-looking', 'adroit', 'artful', 'unconventional', 'ready', 'undimmed', 'burnished', 'lustrous', 'propitious', 'fulgent', 'glimmering', 'brightly', 'luminiferous', 'innovatory', 'innovational', 'gifted', 'ingenious', 'deeply', 'able', 'fertile', 'knowledgeable', 'bass', 'inscrutable', 'agog', 'trench', 'recondite', 'sagacious', 'inquisitive', 'versed', 'insightful', 'nonconforming', 'analytical', 'analytic', 'perspicacious', 'introspective', 'important', 'meditative', 'contemplative', 'perceptive', 'artic

tokenizing: 1it [00:00, 2744.96it/s]
tokenizing: 1it [00:00, 3179.91it/s]
tokenizing: 1it [00:00, 2837.82it/s]
tokenizing: 1it [00:00, 2235.77it/s]
tokenizing: 1it [00:00, 2356.35it/s]
tokenizing: 1it [00:00, 2089.84it/s]
tokenizing: 1it [00:00, 2030.16it/s]
tokenizing: 1it [00:00, 1903.91it/s]
tokenizing: 1it [00:00, 2818.75it/s]
tokenizing: 1it [00:00, 2073.31it/s]
tokenizing: 1it [00:00, 2678.36it/s]
tokenizing: 1it [00:00, 2764.87it/s]
tokenizing: 1it [00:00, 2637.93it/s]
tokenizing: 1it [00:00, 2284.48it/s]
tokenizing: 1it [00:00, 2094.01it/s]
tokenizing: 1it [00:00, 2283.24it/s]
tokenizing: 1it [00:00, 2325.00it/s]
tokenizing: 1it [00:00, 2108.75it/s]
tokenizing: 1it [00:00, 2739.58it/s]
tokenizing: 1it [00:00, 2399.49it/s]
tokenizing: 1it [00:00, 2211.02it/s]
tokenizing: 1it [00:00, 2008.77it/s]
tokenizing: 1it [00:00, 2290.72it/s]
tokenizing: 1it [00:00, 2626.36it/s]
tokenizing: 1it [00:00, 2105.57it/s]
tokenizing: 1it [00:00, 1811.79it/s]
tokenizing: 1it [00:00, 2015.52it/s]
t

Top 200 positive label words are: ['aflutter', 'bad-tempered', 'sad', 'overjealous', 'important', 'touchy', 'moody', 'Helen Newington Wills', 'glowering', 'Dwight Lyman Moody', 'easy', 'thin-skinned', 'sadistic', 'ticklish', 'temperamental', 'emotional', 'worked up', 'nettlesome', 'effusive', 'irritable', 'stressed', 'jealous', 'green with envy', 'justificatory', 'ineffectual', 'envious', 'covetous', 'overcautious', 'possessive', 'genitive', 'raring', 'fidgety', 'fretful', 'querulous', 'impatient', 'selfish', 'agent', 'macrophobic', 'self-pitying', 'nervous', 'female', 'emasculated', 'high-strung', 'ratty', 'crabby', 'queasy', 'skittish', 'defensive', 'truculent', 'cantankerous', 'grumpy', 'surly', 'grouchy', 'insecure', 'frightening', 'unsafe', 'cranky', 'dangerous', 'faultfinding', 'anxious', 'frightened', 'unquiet', 'hardline', 'disturbed', 'fearful', 'uneasy', 'quarrelsome', 'captious', 'frightful', 'angstful', 'womanly', 'feminine', 'aware', 'afraid', 'nesh', 'about', 'stupid', 's